### Physics-068
### Project 4: Time Dependent Schrödinger Equation
### Spring 2020
#### Henry Adair, Ben Miller, Kai Stewart

The goal of this project is to simulate the TDSE with the following definition:

$-\frac{1}{2}\frac{\partial^2 \Psi}{\partial x^2} + V\Psi = i\frac{\partial \Psi}{\partial t}$

with the convention that $\hbar = m = 1$

We begin by implementing a finite-difference scheme whereby the differential operators are ultimately combined into a single [tri-diagonal matrix](https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm) for which a system of equations can be solved. Here, the index $n$ corresponds to time and $j$ corresonds to position.

First, convert the individual differential operators using the finite difference scheme.
$\bigg[\frac{\Psi_{j+1}^{n+1} - 2\Psi_{j}^{n+1} + \Psi_{j-1}^{n+1}}{(\Delta x^2)}\bigg] + V_j\Psi_j^{n+1} = i\bigg[ \frac{\Psi_j^{n+1} - \Psi_j}{\Delta t}\bigg]$


$\bigg[\frac{\Psi_{j+1}^{n+1} - 2\Psi_{j}^{n+1} + \Psi_{j-1}^{n+1}}{(\Delta x^2)}\bigg] + V_j\Psi_j^{n+1} = \frac{i\Psi_j^{n+1} - i\Psi_j}{\Delta t}$

Isolate the term corresponding to the starting position $j$ of the wavepacket.
$-\frac{\Delta t}{i}\bigg[\frac{\Psi_{j+1}^{n+1} - 2\Psi_{j}^{n+1} + \Psi_{j-1}^{n+1}}{(\Delta x^2)} + V_j\Psi_j^{n+1}\bigg] = - \Psi_j^{n+1} + \Psi_j$

This last formulation determines how a wavepacket at position $j$ with finite wavenumber $k$ will propagate forwards in time.
$\Psi_j^{n+1} + i\Delta t\bigg[\frac{\Psi_{j+1}^{n+1} - 2\Psi_{j}^{n+1}+ \Psi_{j-1}^{n+1}}{(\Delta x^2)} + V_j\Psi_j^{n+1}\bigg] = \Psi_j$

### Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.linalg import solve_circulant
import imageio
import random
%matplotlib inline

# Functions

### Wave Plotting Function
This function is used to generate plots of the wavefunction in order to create the final animation

In [ ]:
def plot_waves(x_range, psi, V, packet, idx, options, rainbow=False):
    '''
        Plot Waves Function
        Args:
            - x (np_array): NumPy array defining the distance over which the 
                            corresponding wave should be plotted
            - psi (np_array): NumPy array corresponding to a computed psi function
            - V (numpy array): array of potential values
            - packet (dict): initial wavepacket parameters
            - idx (int): index of the loop that is calling this function
            - options (list): options corresponding to the type of simulation that was done
                              - index 0 (str): potential type (linear, harmonic, triangular)
                              - index 1 (str): operator type (tri, circ)
            - rainbow (bool): skittles
        Returns:
            - image for .gif generation
    '''
    fig, ax = plt.subplots(figsize=(20,10))
    
    if rainbow:
        c = cm.rainbow(np.linspace(0, 1, 3))
        c_idx = [0,1,2]
        random.shuffle(c_idx)

    ## Plot settings
    offset = packet['N'] * 0.15
    
    # Set axes limits
    ax.set_xlim(min(x_range) + offset, max(x_range) - offset)
    ax.set_ylim(-0.25, 0.50)
    

    # Plot wave on axes object
    if rainbow:
        ax.plot(x_range, psi.real,    color = c[c_idx[0]], label='$\mathcal{Re}(\Psi)$')
        ax.plot(x_range, psi.imag,    color = c[c_idx[1]], label='$\mathcal{Im}(\Psi)$')
        ax.plot(x_range, np.abs(psi), color = c[c_idx[2]], label=f'$\Psi$_{idx}')
        ax.set_facecolor('#000000')
    else:
        ax.plot(x_range, psi.real,    color = 'blue',   label='$\mathcal{Re}(\Psi)$')
        ax.plot(x_range, psi.imag,    color = 'orange', label='$\mathcal{Im}(\Psi)$')
        ax.plot(x_range, np.abs(psi), color = 'k',      label=f'$\Psi$_{idx}')
        ax.grid()
    
    # Plot potential on all axes obejcts
    ax.plot(x_range, V, color='purple', linestyle='--', label='Potential')
    
    ax.set(xlabel='Postion [nm]', ylabel=f'$\Psi$_{idx}', title='$\Psi$ Animation')
    ax.legend()
    
    # Set figure title
    if 'harmonic' in options[0]:
        if options[-1] == 'skittles':
            plt.suptitle("Quantum Harmonic Skittles Oscillator", fontsize=16)
            
        else:
            plt.suptitle("Quantum Harmonic Oscillator", fontsize=16)
    elif 'lin' in options[0]:
        plt.suptitle("Wavepacket in Linear Potential", fontsize=16)
    elif 'tri' in options[0]:
        plt.suptitle("Wavepacket in Triangular Potential", fontsize=16)
    elif 'inf' in options[0]:
        plt.suptitle("Wavepacket in Infinite Potential", fontsize=16)
    else:
        plt.suptitle("Free Wavepacket", fontsize=16)
    
    # Used to return the plot as an image array
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()
    
    return image

### Operator Function
This function generates entries for each row of the matrix operator that will act on the wavefunction during the system solving routine.

In [ ]:
def operator(dX, dT, V):
    '''
        Function that generates the operator sequence for each row of the
        matrix used to solve the TDSE.
        Args:
            - dX (float): step size of the spatial coordinateds
            - dT (float): step size of the temporal coordinates
            - V (float): value of the potential function for a given spatial coordinate
        Returns:
            - operator entry (array): list of operators to be included in the operator matrix
    '''
    dtdx = (-1j*dT)/(dX**2)
    d2xdt2 = 1 + dtdx*(-2 + V)
    op = np.empty((3,), dtype=np.complex_)
    op[0] = dtdx
    op[1] = d2xdt2
    op[2] = dtdx
    return op

### Tri-Diagnonal Matrix Function
[Tri-Diagonal Matrix](https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm)

In [ ]:
def gen_tridiag(dX, dT, V, N):
    '''
        Function that creates a tri-diagonal operator
        Args:
            - dX (float): step size of the spatial coordinates
            - dT (float): step size of the temporal coordinates
            - V  (array): values of the potential function in which the particle
                          will be propagated
            - N  (int): dimension of the matrix corresponding to the length of
                        the array that represents the wavepacket. Must be geq 3.
        Returns:
            - A (Numpy Array): [nxn] tri-diagonal finite difference operator 
    '''
    # Declare an array of zeros with appropriate dimensions
    A = np.zeros((N,N), dtype = np.complex_)
    
    # Place operator entries in locations b1 and c1
    op = operator(dX, dT, V[0])
    A[0][0:2] = op[1:3]
        
    # Place operator entries in locations ai+1, bi+1, and ci+1
    for i in range(1,N-1):
        A[i][i-1:i+2] = operator(dX, dT, V[i])
    
    # Place operator entries in locations bn and cn
    op = operator(dX, dT, V[-1])
    A[-1][-2:] = op[0:2]

    return A

### Circulant Matrix Function
[Circulant Matrix](https://en.wikipedia.org/wiki/Circulant_matrix)

In [ ]:
def gen_circ(dX, dT, V, N):
    '''
        Function that creates a circulant matrix operator
        Args:
            - dX (float): step size of the spatial coordinates
            - dT (float): step size of the temporal coordinates
            - V  (array): values of the potential function in which the particle
                          will be propagated
            - N  (int): dimension of the matrix corresponding to the length of
                        the array that represents the wavepacket. Must be >= 3.
        Returns:
            - A (Numpy Array): [nxn] circular finite difference operator 
    '''
    # Declare an array of zeros with appropriate dimensions
    A=np.zeros((N,N), dtype=np.complex_)

    op = operator(dX, dT, V[0])
    # Place operator entry in top right corner
    A[0][-1] = op[0]
    # Place operator entries in locations b1 and c1
    A[0][0:2] = op[1:3]

    # Place operator entries in locations ai+1, bi+1, and ci+1
    for i in range(1,N-1):
        A[i][i-1:i+2] = operator(dX, dT, V[i])
    
    op = operator(dX, dT, V[-1])
    # Place operator entry in lower left corner
    A[-1][0] = op[2]
    # Place operator entries in locations bn and cn
    A[-1][-2:] = op[0:2]

    return A

### Operator Matrix Generation Function

In [ ]:
def GenMatrix(dX, dT, V, N, option):
    '''
        Function that creates either a tri-diagonal or circulant matrix
        Args:
            - dX (float): step size of the spatial coordinateds
            - dT (float): step size of the temporal coordinates
            - V  (array): values of the potential function in which the particle
                          will be propagated
            - N  (int): dimension of the matrix corresponding to the length of
                        the array that represents the wavepacket. Must be >= 3.
            - option (str): determines which kind of matrix the function should return,
                            either Tri-Diagonal or Circulant.
                            - Arguments are: 'tri', 'circ'.
                            - (default) 'tri'
        Returns:
            - A (Numpy Array): [nxn] finite difference operator 
    '''
    assert N>=3, "The dimension of the operator must be greater than or equal to 3"
    
    if 'circ' in option:
        return gen_circ(dX, dT, V, N)
    elif 'tri' in option:
        return gen_tridiag(dX, dT, V, N)
    else:
        print('No operator geometry selected - defaulting to tridiagnonal implementation')
        return gen_tridiag(dX, dT, V, N)

### Wavepacket Generation Function
[Gaussian Wavepacket Localized in k-space](https://quantummechanics.ucsd.edu/ph130a/130_notes/node80.html)

In [ ]:
def wavepacket(alpha, k, center, N, step, dx, dt):
    '''
        Function that constructs a Gaussian wavepacket for a given wavenumber.
        Args:
            - alpha  (float): Variance of the wavepacket
            - k      (float): Mean wavenumber of the wavepacket
            - center (float): specify where the wavefunction should be centered
            - N      (int):   Number of steps over which the function should be defined
            - step   (float): Size of the steps between each point in the wavepacket
        Returns:
            - psi (Numpy array): corresponds to a Gaussian wave-packet localized in k.
    '''
    # Define the normalization constant
    c = (1 / (2 * alpha * np.pi))**(1/4)
    
    # Define the range of k values for the wavepacket
    p = np.linspace(-N, N, step)
    
    # Construct the wavepacket 
    ## Complex component
    psi_im = np.exp(1j * k * (p + center))
    
    ## Real component
    psi_re = np.exp( -np.square(p + center) / (4 * alpha) )
    
    return c * psi_im * psi_re

### Potential generation function

In [ ]:
def potential(x_range, steps, dx, option = None):
    '''
        Function that generates an array corresponding to a 1D potential
        Args:
            - steps (int): number of steps over which the potential should be defined
            - x_range (numpy array): spatial range of the wavepacket
            - dx (float): step size that scales the potential function
            - option (str): selects for a particular potential configuration.
                            - Arguments are:
                                - linear
                                - harmonic
                                - triangular
                                - infinite
                            (default): zero potential
        Returns:
            Numpy array corresponding to the selected potential
    '''
    if 'lin' in option:
        V1 = np.array([0 for i in range(0, len(x_range)//2)])
        V2 = np.array([i for i in range(0, len(x_range)//2)])
        return np.concatenate((V1, V2)) * dx
    elif 'har' in option:
        limit = int(len(x_range) * 0.10)
        V_right = np.array([(1/16)*i**2 for i in range(0, len(x_range)//2)])
        V_right[limit:] = 10e10
        V_left  = V_right[::-1]
        return np.concatenate((V_left, V_right)) * dx
    elif 'tri' in option:
        V2 = np.array([i for i in range(0, len(x_range)//2)])
        V1 = V2[::-1]
        return np.concatenate((V1, V2)) * dx
    elif 'inf' in option:
        limit = int(len(x_range) * 0.50)
        wall = np.array([0 for i in range(0, len(x_range))]) * dx
        wall[len(wall)//2 - limit:] = 10e10 
        wall[len(wall)//2 + limit:] = 10e10
        return wall 
    elif option is None:
        print('No potential well configuration selected - defaulting to zero potential')
        return np.array([0 for i in range(0, len(x_range))]) * dx

### Wrapper Function for Computing Experiment

In [ ]:
def schroedingers(cat):
    if cat:
        print("Ze cat is alive =^_^=")
    else:
        print("Ze cat is dead =x_x=")

In [ ]:
def compute_experiment(wave_packet, options, gif=False):
    '''
        Function that wraps up the TDSE solver and allows for the seletion of various
        potentials and matrix operator configurations.
        Args:
            - wave_packet (Numpy Array): The initial system wave packet
            - options (list): options corresponding to the type of simulation to be carried out
                              - index 0 (str): potential type (linear, harmonic)
                              - index 1 (str): operator type (tri, circ)
        Returns:
            - waves (list): numpy arrays corresponding to the solutions of the system
                            for n steps as specified by the length of the intial wavepacket array
            - images (list): matplotlib image array objects that can be converted to .gif using imageio
                             or plotted using matplotlib itself.
    '''
    print(f"Computing TDSE solver for a {options[0]} potential using a {options[1]} matrix operator\n")
    
    ## Storage for numerical results
    waves = []; images = []
    
    ## Declare the x_range over which we're computing
    x_range = np.linspace(-wave_packet['N'], wave_packet['N'], wave_packet['step'])
    
    ## Generate the values of the potential
    V = potential(x_range, wave_packet['N'], 0.001, option=options[0])
    
    ## Generate the wavepacket
    print(f"Generating wavepacket")
    psi = wavepacket(**wave_packet)

    ## Generate the operator matrix
    A = GenMatrix(wave_packet['dx'], wave_packet['dt'], V, wave_packet['step'], option = options[1])
    
    ## Compute the first step of the system
    print("Solving system\n")
    Ax = np.linalg.solve(A, psi)
    waves.append(Ax)
    
    ## Compute the next n-steps
    for i in range(len(psi)-1):
        Ax = np.linalg.solve(A, Ax)
        waves.append(Ax)
    
    ## Generate images of experimental results
    print(f"Generating images of {len(waves)} results\n")
    if options[-1] == 'skittles':
        for idx, wave in enumerate(waves):
            images.append(plot_waves(x_range, wave, V, wave_packet, idx, options, rainbow=True))
    else:
        for idx, wave in enumerate(waves):
            images.append(plot_waves(x_range, wave, V, wave_packet, idx, options))
    
    ## Generate a .gif file, if desired
    if gif:
        print("Generating .gif file\n")
        if options[-1] == 'skittles':
            imageio.mimsave(f'./{options[0]}-{options[1]}-{options[2]}.gif', images, fps=30.0)
        else:
            imageio.mimsave(f'./{options[0]}-{options[1]}.gif', images, fps=30.0)

    cat = np.random.randint(0, 2, size=1)
    schroedingers(cat)
    
    return waves, images

----
# Experiment

### Set up a wavepacket and simulation parameters

In [ ]:
# Space and time step sizes
dx = dt = 0.1

# Wave packet config
wave_packet = {'alpha': 1, 'k': 0, 'N': 50, 'center': 0}
wave_packet.update( {'step': int( (wave_packet['N'] + wave_packet['N']) / dx) } )
wave_packet.update( {'dx':dx, 'dt':dt } )

### Compute the desired experimental configuration
Select a configuration using the options list:

options = ['potential', 'matrix operator type']
    - potentials: flat, linear, harmonic, triangular, infinite
    - matrix operator type: tridiagonal, circulant
    
----

In [ ]:
_, _ = compute_experiment(wave_packet, options=['triangular', 'tridiagonal'], gif=True)

----